# Running Async UWS Jobs via the uws-client library

## Install uws-client 

In [ ]:
!pip install --upgrade --user git+https://github.com/stvoutsin/uws-client.git

## Import requirements

In [ ]:
from uwsclient import UWSClient
from uwsclient.models import UWSPhase
from lsst.rsp.utils import get_access_token

## Create client instance

In [ ]:
client = UWSClient(
    base_url="https://data-dev.lsst.cloud/api/cutout",
    token=get_access_token()
)

## Setup job params

In [ ]:
query_params = {
    "circle": "55.7467 -32.2862 0.05",
    "id": "butler://dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c"
}

## Create and submit job

In [ ]:
job_id = client.create_job(query_params)
print(f"Created job: {job_id}")

## Wait for completion

In [ ]:
status = client.wait_for_job_completion(job_id, timeout=300)
print(f"Final status: {status['phase']}")

## Download results if successful

In [ ]:
if status['phase'] == UWSPhase.COMPLETED.value:
    results = client.get_job_results(job_id)
    for i, result in enumerate(results):
        if result.get('href'):
            output_path = f"cutout_result_{i}.fits"
            client.download_result(result['href'], output_path)
            print(f"Downloaded result to {output_path}")


## Close client

In [ ]:
client.close()